<img align="left" src="https://ithaka-labs.s3.amazonaws.com/static-files/images/tdm/tdmdocs/tapi-logo-small.png" />

This notebook free for educational reuse under [Creative Commons CC BY License](https://creativecommons.org/licenses/by/4.0/).

Created by [Firstname Lastname](https://) for the 2023 Text Analysis Pedagogy Institute, with support from [Constellate](https://constellate.org).

For questions/comments/improvements, email author@email.address.<br />
____

# Taming Text 2

This is lesson `2` of 3 in the educational series on Taming Text. This notebook is intended to introduce XPath and includes some basic review of XML etc. 

**Audience:** `Teachers` / `Learners` / `Researchers`

**Use case:** `Tutorial` / `How-To` 


**Difficulty:**  `Intermediate` / `Advanced`

**Completion time:** `90 minutes`

**Knowledge Required:** 


* Python basics (variables, flow control, functions, lists, dictionaries)


**Knowledge Recommended:**


**Learning Objectives:**


**Research Pipeline:**

Can be at many points.

In [ ]:
!pip install lxml


(some of this was already covered during day 1, but the refreshers are here just to help make this document a bit more independent.)

For new to XML, please start here: https://www.w3schools.com/xml/default.asp.  Read Introduction through Attributes, then stop.  Those who've worked with XML should at least take a skim through those pages and refresh your understanding of the XML lingo.

## What is XML?

If you truly have zero knowledge of XML, I invite you to start with the a good skim of the [Wikipedia page](https://en.wikipedia.org/wiki/XML) on the subject. Don't pour over it, but it'll provide some important background vocabulary and context.  Anyhow, XML is ruleset for marking up documents in specific ways, and has been extended to a method of storing data in a very structured way.  Instead of having a row/column structure like a CSV file, you can have nested and thus much more complex data storage this way.

Much of library metadata is stored in XML marked up documents, and that's the focus of the Metadata in Theory and Practice class offered at the iSchool.  Meanwhile, HTML is another markup language that works very similarly to XML.  Unless the HTML is severely malformed, techniques to extract data out of XML will also be useful for extracting data out of web pages.

## What is XPath?

XPath (https://en.wikipedia.org/wiki/XPath) is a query language (a la SQL, kind of) used to describe both locations for items and data extraction for XML documents/data.  This means that you can use it to both locate a specific element within an XML document but it also includes functions to pull out desired values.  Much of the time that's the text of that element, but sometimes you'll want other stuff.

XPath is a system that is platform and tool independent, and thus you can actually find tools for it in the Oxygen XML editor, and there are a few other resources.  There are many Python tools that utilize XPath and have functions for applying XPath queries, but we're going to explore one of those.  

## Installing lxml

For this first week we'll be using some other tools than python for exploring xpath, but it wouldn't hurt to go ahead and get this installed.

Your anaconda installation should already have included an installation of lxml.  Should you need it, lxml is a module available from PyPi, which means you can use pip to install it.  Please follow these directions:

You can use pip in the command line via `pip install lxml`, or alternatively use `!` in jupyter to install it.

We will import the module via `from lxml import etree`

## Xpath tools

There are several tools that will run xpath queries.  Oxygen will work on valid xml files, although it tends to complain pretty hard about most common HTML because the internet is the wild west for validity.

There are other tools, like XPath Helper in chrome that can work.

Also, google sheets has a function called `IMPORTXML(url, xpath)` that will work, but the output is pretty restricted. Still, it may be just enough for what you need. 

# Essential vocabulary

No matter which scraping or parsing tool that you use, you will not be able to navigate the documentation or create new things if you don't know the language behind the purpose.

Let's take this one example:

```HTML
<a href = "http://ischool.illinois.edu/">iSchool</a>
```

This is how you make a hyperlink in HTML.  The bit between the two tags is what shows up on the website and the bit in quotes after the href is where the link will go when you click it.  

HTML can be considered to be a specific form of XML.  Remember that XML is just a set of rules, and HTML is just one of those sets (I think there are purists who would disagree on a few points, because modern web browsers allow you to violate every known rule of XML and still render, but that's not a debate to have here).

Here are the essential names that you need to know:

* element name:  this is `a`, where you see in the <>.  The element contains all the information that you want.  The <> define where certain parts of the element exist.  Don't worry, we'll get into more of that.
* node:  roughly, this is the entire contraption that you see there.  The a element and everything about it and what's in it.
* opening tag: this is the `<a>` piece
* closing tag:  this is the `</a>` piece
* element content or value:  sometimes elements will hold just text, another element, a mix of both, or nothing at all!  The stuff that is between the > and < (so after the opening tag and before the closing tag), is the element's content.
* attributes:  these are key/value pairs that appear inside the opening tag.  You can see this is the href.  
* attribute name:  the thing on the left side of the =.  Much like dictionaries, all attribute names must be unique inside the opening tag.
* attribute value:  the thing on the right side of the =.  This is the URL.  Generally you'll find these in quotes, but not always.

Meanwhile, all valid XML must have a single root element that everything belongs inside.  You can see this in proper HTML, which is the `<html>` tag.  Every other element that you see in this website is a descendant element of that root.  Elements (except for the root element) have a parent element.

```XML
<root>
    <middle>
        <child>stuff</child>
    </middle>
</root>
```

Parent, child, and tree:

`root` is the parent of `middle`, and `middle` is the parent of `child`.  Together these make the tree.  `child` is the descendent of `root`. `root` is the ancestor of `child`.

When you are constructing XPath queries, you'll need to operationalize the patterns and locations that you see into these sorts of terms.  Once you can do that, you can string together the names of things in the XML tree and XPath punctuation to build up your query.

We are going to start by exploring things with xpath helper, just to see the syntax better. Then we'll move into python. 

## XPath syntax

A tag needs no other syntax to be to be a reference for an element, as you can see with our references with `child`

###  talking about tags

* `/` looks only 1 level deep, so only for the children of the parent element
* `//` looks anywhere within the descendants of the parent element
* `tag[position number]`:  index starting at 1, allows you to indicate the "nth" instant of that element.  Example, `a[3]` would be the third a found with that query.
* `*` is a wildcard used to represent "any element", usually used within logic queries

The simplest XPath query is a listing a series of tags, separated by `/`, to describe an exact location in the tree.  For example, in the previous structure, I could access the location of `child` via:

`/root/middle/child`

This should look very similar to a URL or a file path.  The `/` is used in a similar way.

However, sometimes there are multiple elements that you want to match or you don't need/want to specify the full path to that element.  You can use `//` to have the query search at any level of the tree instead of starting at the root.

`//child`

This query would look for the `child` tag at any level in the tree.

### Talking about attributes 

* `@` is used before a name to indicate that you are talking about an attribute name instead of an element name.  For example, `@href` for an `a` element.
	* you'll usually use this inside of logic checks, eg checking if there is a tag with that attribute or checking for an attribute with a specific value

### Referring to a relative location

Sometimes you want to refer to your current location or a location's parent element, whichever that may be. These should be familiar to you if you've done command line!

* `.` indicates the current element
	* you'll usually use this inside of functions
* `..` indicates the parent element of the current element.
	* For example, "find this specific element, but then select the element's parent to it"
	* Often used when you navigating to a sibling element

### Logic and checking for things 

You can check a variety of things about a tag, including the contents, if it has a child tag, if it has an attribute, or the contents of an attribute, etc. Use `tag[logical check goes in here]`.

There are a variety of functions and other boolean checks inside the `[]`.  Listed nicely here (https://www.w3schools.com/xml/xsl_functions.asp).

Common usages:

* `tag[@attribute = 'something']` you can use this to select an element with an attribute that has a specific value.  You cas also reference the current element's content with `.`, so `//p[. = "thing"]` finds all the p elements that have the element text value of exactly "thing".  Warning! This will look deeper into the children of that node.  
* If you want to look at the text value of the current element, you should use `text()` instead of `.`.  So `element[text() = "thing"]`.  You can use `text()` or `.` in any of the logical checks that you want.
* You can use compound boolean checks inside the [], such as `//element[@attribute = "something" and contains(., "another")]` . Connect these boolean checks with `and` or `or`. Take note of keeping punctuation in check, etc.
* `//*[@attr = "something"]` to find any tag that has a class specific attribute value.

Of course, all these things are used to just select the element in question.  From there, you have to extract out what you want. 

## Extracting out content

Most often you'll be able to do selection and extraction in the same query, but sometimes you need to separate things out. For example, select a bunch of elements and then loop over them extracting what you need. 

There are two general things you'll want to get out of the elements (there are more things out there, but these are the most common). These usually go right at the end of your query. 

* a attribute's value
    * you can get this by adding `@attribute` to the end of a query
    * example:  `//a/@href` will get the content of the `href = ` attribute.
* the element's text content (as opposed to an element's child elements, etc.)
    * `text()`  is a function Note the `()` in there, those are required.
	    * Example:  `//a/text()`
* Remember that `/` will only look for immediate children that belongs to the element that you have selected.  Some tools are mare permissive about this, but things like lxml are not.  If there are additional elements in there, such as text in a `<b>` tag, it will grab all the text around that tag, but none of the contents.  
	* For example, in `<p>Hello <b>world</b> humans.</p>`, if you use `//p/text()` you'll only get back "Hello . humans.".
* You can get text at any descendant level under the element that you have selected using the `//` notation before `text()`.  
	* So `//p//text()` would grab all the text in there. 
	* Be careful with this one, things sometimes look odd.

We'll use all of these in our example below, but it can be helpful to copy these and keep them handy.

## 4 steps to writing a query

This is a very iterative process, but it can be helpful to have at least some place to start.  

### 1:  Identify your target

You'll either be dealing with an XML file here or you'll want to view source on the website that you're after.  Some websites have protections put in place to prevent you accessing the content, so there may be oddities.  

For websites, copy one of the things you are after and go into the page source.  Search inside the page code to find where it is.  Chrome developer tools (View>Developer>Developer Tools) allows you to navigate the structure in an easier way.

Take note of where that content is, what the element is, parent element is, any attributes it has, those attribute values, etc.  These are all clues that you can use to help select the proper content.  Remember that you arent just after the content that you want, you are after the element itself and the things around it.

In our case, this content is inside an `a` element that is in a `td` (table data) element in an HTML table.

Goal:  be able to operationalize in formal XML vocabulary where the content is located.

### 2:  Pick your starting place

Often, the first element that you start looking for will determine the approach that you end up having to take.  

You have to find a balance that:

* selects all the things that you want
* not so much other stuff that you have too much noise to clean up

There is where you must know the structure of what you are dealing with, and where to find this balance. Something broad enough that it gets you started, but not something so broad that it brings in too much or doesn't provide value.  This is sort of where the art form comes in, and an experienced eye can help direct you.

Goal:  get as close as possible to your target content without losing any of it in the process.

### 3:  Start drilling down

Once you have the starting place, you will start adding elements and other selectors after it until all that is left are the things that you want.  You'll drill in closer and closer, being careful to not leave anything that you want behind.

There may be situations where you start down a perfectly reasonable path, but with that path there may be no way to uniquely specify what you are after.  This may mean that you need to back up and start from a different place, or sometimes you need to bring the values in to Python proper and use string methods etc to get out what you want.

### 4:  Extract the content that you want out of the node

Your first job is to select the node that has what you want, and then you add some content extraction functions or other selectors to your query.  In the best of worlds this will produce exactly the content that you want and nothing else.  However, you may not be so lucky with the structure that you are given, and you might need to do some further processing (like in python) to further filter out what you want.

### 5:  Check your work

You may have set along several paths to get your answer, and don't worry, there isn't some mythical place that you must discover.  You are done with the first query you find that holds true across all your cases.  Let's look at these two pieces:

1. "first one" there's no value in chasing down every path to try and find some perfect solution.  Don't over-engineer the details of what you are after if those details don't add anything to what you are trying to get at.
2. "holds true across all your cases" You may have found a solution to one of your cases, and so you need to test that solution onto all of your data.  This doesn't imply that you have the capability of doing just that (applying an algorithm on many files or whatever you have) and checking that what you are getting back is what you are after.

## The basic pattern with lxml

The basic pattern that we will be exploring here is this:

1. Read in the document (you may need to run it through something like beautiful soup to clean up the HTML if that's what you're using).
2. Parse that object (either an IO object or string depending on your pattern) into a tree object within lxml
3. Apply you desired XPath things to that tree object.

There will be one and only one way you'll need to do this class, but there are other methods out there that you'll see.  

Our pattern for class will be:

1. Read the XML file with `.read()` to read in the text as a big string.
2. Pass that string the parsing method `.fromstring()` to parse into a tree object. (don't forget that you'll have to import this module)
3. Use the `.xpath()` function on that tree object to execute xpath queries on it.


In [ ]:
from lxml import etree

In [ ]:
infile =  open('hamlet-tei.xml', 'rb') # rb required because of encoding
xml = infile.read() # this will be passed to the parser
infile.close()

We usually use `tree` as a convention to indicate that it is the entire XML tree and not a constituent node.  This is using the `.fromstring()` function from the lxml/etree, which will parse string text into a tree object.

In [ ]:
tree = etree.fromstring(xml) 

Now let's use some xpath!

In [ ]:
results = tree.xpath('//persName/text()')

In [ ]:
results

So this is empty because we have an xml namespace to handle.

```python
results = tree.xpath('//alias:elementwhatever/text()', namespaces={'alias': "URL found in the document goes here"})
```

In [ ]:
ns = {'tei': 'http://www.tei-c.org/ns/1.0'}
results = tree.xpath('//tei:persName/text()', namespaces = ns)

In [ ]:
results

## Our data source

Nice little proof of concept, let's talk about the data source a bit.


We'll be using an XML document of "Hamlet" by Shakespeare.  This is located in the hamlet-tei.xml file.  This is a proper XML file that uses the TEI schema.  https://en.wikipedia.org/wiki/Text_Encoding_Initiative  You will want to read this now so you can understand the basics of what's going on in this file.

The data file has its own attribution, but I grabbed it as a material from this workshop: http://tei.it.ox.ac.uk/Talks/2015-08-maynooth

Take some time exploring this file to better understand the structure.  There's no real need to do a full TEI tutorial for this lesson.  This lesson is not meant to be a tutorial on TEI, we're just using it as example data.

This is a very brief description of the structure of the Hamlet file:

* In `teiHeader`:
    * `fileDesc` node contains information about the provenance of the file and content.
    * `profileDesc/particDesc` node contains information on the characters in the play
    * `profileDesc/settingDesc` node contains setting information for the play
* In `text`:
    * this contains nodes for each act, scene, and passage.
    * each passage is in `sp` elements, with `@who` attributes representing the standardized ID for each speaker (those IDs are defined in the `particDesc` node.  The `speaker` reports out what the original text had for the speaker information, and the `l` elements have the individual lines.
    
There are other details that you will need to explore on you own to get a feel for things.

For now, we're going to go ahead and read in our file and prepare our tree object.  You'll only need to do this once at the top of your script.  After that, you'll just be using the tree object.

## a bit more about the namespaces


Most proper XML files have namespaces (there can be multiple) that you'll need to navigate.  As this is not a metadata or TEI course, I will not provide an extended discussion on what this is.

We can see in line 4 of the document, which has the root element:  `<TEI xmlns="http://www.tei-c.org/ns/1.0">`

This is saying that the elements found in this root node belong to the TEI schema, with a URL to the schema definition. 

The namespace dictionary can have multiple namespaces declared, where the alias is the key (as a string) and the value is the URL (as a string) as seen in the file.  You may use any alias you would like for that namespace, but the URL must perfectly match what appears in the file.

Provide as many alias: URL pairs as you need for your document.  Most namespaces have a canonical alias to use, which you should abide by when possible.


# Evaluating an extraction query to get a single result

Remember that all your previous queries all needed to end with an extraction function at the end.  This was likely either `/text()` to get the text of the element out, or `@attribute` to get some attribute text out.

For example, `//a` would select all the `a` element nodes, but not yield the contents.  But `//a/text()` would give you the hyperlink text, and `//a/@href` would give you all the URLs for the hyperlinks.

The reasons for this aren't always made clear by those GUI tools we were using.  However, the distinction between a selection and extraction query will be striking when using this tool in Python.  You **must** include an extraction statement in your query to get text content out.  Otherwise you'll be selecting elements, and nothing will appear useful in the list of results.

As a start, we're going to run a query that will extract out a single result.  We're going to look up the standard name of Hamlet from his character data node.

The xpath that we would want to use is `//person[@xml:id = "F-ham-ham"]/persName[@type = "standard"]/text()`, but we need to adapt this to our namespace.  Look back up to our `ns` dictionary and look at what we declared the alias to be.  We gave our TEI namespace schema an alias of `tei`, which means we need to provide this before each element name that we are referencing.  IMPORTANT! You only need to do this for element names, not for attribute values, content, or XPath functions.  Literally only for the element names, but for **every** element name. Even when you have multiple.

So now our new XPath query with correct aliases will be:

`//tei:person[@xml:id = "F-ham-ham"]/tei:persName[@type = "standard"]/text()`

See those `tei:person` and `tei:persName`?  That's how you use that alias value.  It's `alias:element`.

Let's put this together and see the results.

In [ ]:
print(tree.xpath('//tei:person[@xml:id = "F-ham-ham"]/tei:persName[@type = "standard"]/text()', namespaces = ns))

Things to note: 

* I am using my alias here only for the elements, and that alias name matches what I have declared in my `ns` object.
* I have `namespaces = ns` which will need to be in **each and every xpath query you run for this assignment**.
* my xpath query is just a string
* I've used double quotes in my xpath query, which means that I need to use single quotes to surround the string.
* my results are coming back as a list with one element.  I know and expect there to be just a single result, but the results will always be coming back to you as a list.
* that extra text is from a the newline in the XML file itself.

# Query to extract many results

Let's adapt our previous result to find all the standard names for these characters.  We don't need to change much.  We need to take out the `@xml:id = "F-ham-ham"` that selected only Hamlet's node, and now it will select all the person nodes.

In [ ]:
results = tree.xpath('//tei:person/tei:persName[@type = "standard"]/text()', namespaces = ns)
print(results)

print(len(results)) # how many characters have a standard name?

# clean up the spaces in the names
for name in results:
    print(" ".join(name.split()))

Profiling structures

You can't be an expert in all schemas, so sometimes you need to use some tools in python to profile the data that you are working with.

We can look inside the Hamlet person node and see that there are 4 reported variations:

``` XML
<persName type="form">Ha.</persName>
<persName type="form">Ham.</persName>
<persName type="form">Hamlet.</persName>
<persName type="form">Hem.</persName>
```

But can we confirm that this really is the case?  Alternatively, what if we were the ones writing this data file and needed to fill this in?  Also, this doesn't include the counts, so we don't really know the distribution of these forms.  

Let's write a query that finds all the speaker representations of Hamlet, and then runs the results through the couter tool that we've seen before.

Here's our xpath to find all of Hamlet's passages:

`//tei:sp[@who = "#F-ham-ham"]`

Now find all the speaker elements in there.

`//tei:sp[@who = "#F-ham-ham"]/tei:speaker`

Now get all that text out!

`//tei:sp[@who = "#F-ham-ham"]/tei:speaker/text()`

In [ ]:
results = tree.xpath('//tei:sp[@who = "#F-ham-ham"]/tei:speaker/text()', namespaces = ns)
print(results)
print(len(results))

In [ ]:
from collections import Counter

print(Counter(results))

## Selecting elements and looping over them

Up to now, we've been focusing on the extraction of data.  However, this tool is much more powerful than that.  As we've discussed with other data structures in previous lectures, sometimes it can be really valuable to isolate the specific data granularity that you want.  Once you have those chunks isolated, you can drill down into them to get out information that you want.  We can do the same thing here.

The value of being able to select just a node (instead of extracting information out of it) is that you can save that object node as a variable and apply xpath queries directly onto it.  Yes, we could always include that information in our original xpath if we were wanting a single value.  But sometimes we want more.

However, when we can isolate a node we can run however many xpath queries we want on that node.  And this is why it is powerful.

Some of the examples that we will be going through below could also be done with xpath functions, but those aren't always supported inside these packages.  Also, this lesson is meant to highlight brining in data into python.

So with that said, let's explore this.

You can easily select just the nodes for your query by omitting the extraction chunk of your query.

We're curious about stage directions in speaker elements.  These directions have both text that we want and attribute values.  We could write this in two queries.

In [ ]:
stagedirtype = tree.xpath('//tei:sp/tei:stage/@type', namespaces=ns)
stagedirtext = tree.xpath('//tei:sp/tei:stage/text()', namespaces=ns)
print(len(stagedirtype))
print(len(stagedirtext))

Hmmm, so if we did this with two queries, we can see that there are differing length results.  This means that the results don't line up via positions, and there aren't ways that I can predict or know by just looking at the content.  So doing this as two separate passes won't work.

And indeed, there are some stage elements that do not have type attributes.  Example:  `<stage rend="italic inline">within.</stage>`

Using this structure of gathering all the elements and then extracting the content allows us to navigate this kind of situation and provides protection when we might not expect that to be the case.

Let's rewrite our query such that we only get stage elements that have the type attribute (of any value).  This time, we're only going to select the matching elements, and not extract anything.

To check that an element contains an attribute value, we can place that attribute reference in the logical check area, but with no operators.  We can select and gather all the elements by omitting any extraction notations on the end.

In [ ]:
stagedirswithtype = tree.xpath('//tei:sp/tei:stage[@type]', namespaces=ns)

And when we print this out, we don't see text.  We see that we are storing objects in memory that have a nice method of printing (because what would you print?). That's what that <> thing means around them.  We have Element objects stored, but we are getting the default string representation.  

While this might look like an error, it is exactly what we want!

We now have a list of objects, and we want to loop over them.  Well, we don't need anything fancy for that.

In [ ]:
# kinda gross looking
for stage in stagedirswithtype:
    print(stage.xpath('../@who', namespaces = ns))
    print(stage.xpath('@type', namespaces = ns), stage.xpath('text()', namespaces = ns))

In [ ]:
# prettier
for stage in stagedirswithtype:
    who = stage.xpath('../@who', namespaces = ns)
    dirtype = stage.xpath('@type', namespaces = ns)
    dirtext = stage.xpath('text()', namespaces = ns)
    dirtextclean = " ".join(dirtext[0].split())
    print(who, dirtype, dirtextclean)

In [ ]:
# collect everything up for a data file

allresults = []

for stage in stagedirswithtype:
    results = []
    who = stage.xpath('../@who', namespaces = ns)
    dirtype = stage.xpath('@type', namespaces = ns)
    dirtext = stage.xpath('text()', namespaces = ns)
    results.append(who[0])
    results.append(dirtype[0])
    results.append(" ".join(dirtext[0].split()))
    allresults.append(results)

In [ ]:
allresults

# Exercises (Optional)

`If possible, include practice exercises for users to do on their own. These may have clear solutions or be more open-ended.`

# Solutions (Optional)
`Offer some possible solutions for the practice exercises.`


# References (Optional)
No citations required but include this if you have cited academic sources. Use whatever format you like, just be consistent. Markdown footnotes are not well-supported in notebooks.[$^{1}$](#1) I suggest using an anchor link with plain html as shown.[$^{2}$](#2)

1. <a id="1"></a> Here is an anchor link footnote.
2. <a id="2"></a> D'Ignazio, Catherine and Lauren F. Klein. [*Data Feminism*](https://mitpress.mit.edu/books/data-feminism). MIT Press, 2020.